# Import all Keras, WandB, Pillow, and FastAPI Libraries

## Import Keras, FastAPI, Pillow, time, and numpy libraries

In [1]:
#%%capture
!pip install fastapi nest-asyncio pyngrok uvicorn python-multipart
import keras
from keras import models, preprocessing, datasets, Model, losses

import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from PIL import Image

import time

import numpy as np

from fastapi import FastAPI, File, UploadFile
import nest_asyncio
from pyngrok import ngrok
import uvicorn

from io import BytesIO

     |████████████████████████████████| 52 kB 1.1 MB/s eta 0:00:011
     |████████████████████████████████| 745 kB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 8.3 MB/s  eta 0:00:01
     |████████████████████████████████| 60 kB 6.9 MB/s  eta 0:00:01
     |████████████████████████████████| 54 kB 4.9 MB/s  eta 0:00:01
  Created wheel for pyngrok: filename=pyngrok-5.0.5-py3-none-any.whl size=19246 sha256=4ad47194a6113db537dc2b0f5decbe5464abbe14570154a17a57d1a11cd6f962
  Stored in directory: /home/jupyter/.cache/pip/wheels/1b/f7/72/35c95a53d15b91dd00df6cf1304d49a31ec5ed6f954c2d4e32
  Created wheel for python-multipart: filename=python_multipart-0.0.5-py3-none-any.whl size=31670 sha256=aff72f9899e69081a911434afd937b4e2511326fcc611efeb3f5e54aab96c921
  Stored in directory: /home/jupyter/.cache/pip/wheels/2c/41/7c/bfd1c180534ffdcc0972f78c5758f89881602175d48a8bcd2c
Successfully built pyngrok python-multipart


Using TensorFlow backend.


## Import Weights and Balances Python Library

In [2]:
#%%capture
!pip install wandb
import wandb

     |████████████████████████████████| 1.8 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 9.6 MB/s  eta 0:00:01
     |████████████████████████████████| 133 kB 23.6 MB/s eta 0:00:01
  Created wheel for subprocess32: filename=subprocess32-3.5.4-py3-none-any.whl size=6488 sha256=ad1bb535c5211c6b26c02c5e03234c2c164e67faac97b0b930825789e5157efd
  Stored in directory: /home/jupyter/.cache/pip/wheels/50/ca/fa/8fca8d246e64f19488d07567547ddec8eb084e8c0d7a59226a
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8784 sha256=1e1798ddf1f4cece5d53be81d403b2e40fa28c89057045eb03dff1442ccdda6f
  Stored in directory: /home/jupyter/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built subprocess32 pathtools


# Log in to Weights and Balances (Free Account Available to Try)

In [4]:
wandb.login()

wandb: Currently logged in as: jamesysato (use `wandb login --relogin` to force relogin)


True

# Set up System Parameters

## Setup Google Cloud Project and Model Location

In [5]:
CLOUD_PROJECT = 'mlops-content1' # Cloud Project Name
BUCKET = 'gs://' + CLOUD_PROJECT + '-james-mlops-midterm' # Model Storage Bucket
model_dir = 'james-mlops-midterm'

## Initialize Google Cloud with Parameters

In [6]:
!gcloud config set project $CLOUD_PROJECT

Updated property [core/project].


# Check your Cloud Bucket for the model files



In [7]:
!gsutil ls -r $BUCKET/$model_dir

gs://mlops-content1-james-mlops-midterm/james-mlops-midterm/:
gs://mlops-content1-james-mlops-midterm/james-mlops-midterm/
gs://mlops-content1-james-mlops-midterm/james-mlops-midterm/keras_metadata.pb
gs://mlops-content1-james-mlops-midterm/james-mlops-midterm/saved_model.pb

gs://mlops-content1-james-mlops-midterm/james-mlops-midterm/assets/:
gs://mlops-content1-james-mlops-midterm/james-mlops-midterm/assets/

gs://mlops-content1-james-mlops-midterm/james-mlops-midterm/variables/:
gs://mlops-content1-james-mlops-midterm/james-mlops-midterm/variables/
gs://mlops-content1-james-mlops-midterm/james-mlops-midterm/variables/variables.data-00000-of-00001
gs://mlops-content1-james-mlops-midterm/james-mlops-midterm/variables/variables.index


# Load Model

In [8]:
model = tf.keras.models.load_model(BUCKET + f'/{model_dir}')
model.summary()

Model: "sato_4B_MLOps_2021_June_midterm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
activation (Activation)      (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None,

# Set Up Prediction Workflow

## Input Normalization

In [9]:
# Production Time Standardized normalization based upon training time values
def normalize_production(x):
    # This function is normalizes instances in production according to saved training set statistics
    # Input: X - a training set
    # Output X - a normalized training set according to normalization constants.

    # Mean and Standard Deviation values of dataset saved from training time
    mean = 121.98703
    std = 68.42943
    #these values produced during first training and are general for the standard cifar10 training set normalization
    return (x-mean)/(std+1e-7)

## Prediction Function

In [10]:
# Prediction Function
def predict(x):
    prediction = model.predict(x)
    output = np.argmax(prediction, axis=-1)
    return output

## Class Names for the CIFAR100 Dataset that Model Outputs to

In [11]:
CLASS_NAMES = ["apple", "aquarium_fish", "baby", "bear", "beaver", "bed", "bee", "beetle", "bicycle", "bottle", "bowl", "boy", "bridge", "bus", "butterfly", "camel", "can", "castle", "caterpillar", "cattle", "chair", "chimpanzee", "clock", "cloud", "cockroach", "couch", "crab", "crocodile", "cup", "dinosaur", "dolphin", "elephant", "flatfish", "forest", "fox", "girl", "hamster", "house", "kangaroo", "keyboard", "lamp", "lawn_mower", "leopard", "lion", "lizard", "lobster", "man", "maple_tree", "motorcycle", "mountain", "mouse", "mushroom", "oak_tree", "orange", "orchid", "otter", "palm_tree", "pear", "pickup_truck", "pine_tree", "plain", "plate", "poppy", "porcupine", "possum", "rabbit", "raccoon", "ray", "road", "rocket", "rose", "sea", "seal", "shark", "shrew", "skunk", "skyscraper", "snail", "snake", "spider", "squirrel","streetcar", "sunflower", "sweet_pepper", "table", "tank", "telephone", "television", "tiger", "tractor", "train", "trout", "tulip", "turtle", "wardrobe", "whale", "willow_tree", "wolf", "woman", "worm"]

## Weights and Biases Logging Variables

In [12]:
# Track historical outputs for entire deployment
output_hist = np.zeros((100, 1))

# Track Average Output Value for Data Skew
output_val_hist = []
output_count = 0

# Track Historical Prediction Time for entire deployment
pred_hist = []

## Prediction Runtime with WandB Logging

In [13]:
# Prediction Runtime Function
def run_predict_single(image: Image.Image):
    input = np.asarray(image.resize((32, 32)))[..., :3]
    input = normalize_production(input)
    #input = tf.image.resize(input, (32, 32))
    input = np.expand_dims(input, axis = 0)
    input = tf.convert_to_tensor(input)
    
    start_time = time.time()
    output = predict(input)
    end_time = time.time()

    predict_time = end_time - start_time


    # Increase Output Count by 1
    global output_count
    output_count = output_count+1
    output_val_hist.append(output)

    # Increase output count to historical outputs
    output_hist[output] = output_hist[output] + 1
    
    # Calculate Output with greatest number of appearances
    max_count = np.argmax(output_hist)

    # Calculate Output History Average
    output_ave = sum(output_val_hist)/output_count

    # Add Prediction time to history
    pred_hist.append(predict_time)
    pred_time_ave = np.mean(pred_hist)

    # Save Integer Output for WandB logging
    output_num = output

    output = CLASS_NAMES[output[0]]
    output = str(output)
    wandb.log(
        {'Prediction Time': predict_time, 'Average Prediction Time': pred_time_ave, 'Output': output_num, 'Average Output': output_ave, 'Most Occurring Output': max_count}
    )
    #return input.shape
    return output

## Image File Reading Function

In [14]:
def read_imagefile(file) -> Image.Image:
    image = Image.open(BytesIO(file))
    return image

# Create FastAPI App

In [15]:
app = FastAPI()

@app.on_event("startup")
def start_wandb():
   wandb.init(project="mlops-midterm", sync_tensorboard=True)
   return {'message': 'Weights and Balances Started'}


@app.get('/')
def index():
    return {'message': 'This is the homepage of the model, add \'/docs\' to the end of the URL to access FastAPI to make predictions with the model'}

@app.get('/reload_model')
def reload_model():
   global model
   model = model = tf.keras.models.load_model(BUCKET + f'/{model_dir}')
   return {'message': 'Model on GCP reloaded at ' + BUCKET + '/' + model_dir}

@app.get('/change_model')
async def change_model(string_input):
   global model_dir
   model_dir = str(string_input)
   model = tf.keras.models.load_model(BUCKET + f'/{model_dir}')
   return {'message': 'Model on GCP loaded from ' + BUCKET + '/' + model_dir, 'warning': 'This function doesn\'t currently check for file existence at location'}

@app.post('/predict_single')
async def predict_api(file: UploadFile = File(...)):
    extension = file.filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not extension:
        return "Image must be jpg or png format!"
    image = read_imagefile(await file.read())
    prediction = run_predict_single(image)
    prediction = str(prediction)
    print(prediction)
    return prediction


# Run Application

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://5ccc83811cfb.ngrok.io                                                            


INFO:     Started server process [405]
INFO:     Waiting for application startup.


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     209.143.6.229:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     209.143.6.229:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     99.147.232.13:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     99.147.232.13:0 - "GET /openapi.json HTTP/1.1" 200 OK
keyboard
INFO:     99.147.232.13:0 - "POST /predict_single HTTP/1.1" 200 OK
keyboard
INFO:     99.147.232.13:0 - "POST /predict_single HTTP/1.1" 200 OK
keyboard
INFO:     99.147.232.13:0 - "POST /predict_single HTTP/1.1" 200 OK
keyboard
INFO:     99.147.232.13:0 - "POST /predict_single HTTP/1.1" 200 OK
keyboard
INFO:     99.147.232.13:0 - "POST /predict_single HTTP/1.1" 200 OK
keyboard
INFO:     99.147.232.13:0 - "POST /predict_single HTTP/1.1" 200 OK
keyboard
INFO:     99.147.232.13:0 - "POST /predict_single HTTP/1.1" 200 OK
keyboard
INFO:     99.147.232.13:0 - "POST /predict_single HTTP/1.1" 200 OK
keyboard
INFO:     99.147.232.13:0 - "POST /predict_single HTTP/1.1" 200 OK
keyboard
INFO:     99.147.232.13:0 - "POST /predict_single HTTP/1.1" 200 OK
